In [1]:
import pandas as pd
import numpy as np
from math import floor
import math
import os
from datetime import datetime
import xarray as xr

# Listado de playas con avistamientos y fechas

In [2]:
avistamientos_df = pd.read_excel("./Physalia_Ambiental_R.xlsx")
# avistamientos_df = pd.read_excel("../Physala_Data/Datos_Physalia_20171010.xls")

columnas = avistamientos_df.iloc[0]

#Quito las 3 primeras filas, debido al formato de la excell
avistamientos_df = avistamientos_df.iloc[3:] 
avistamientos_df.columns = columnas

# Me quedo solo con los datos de avistamientos
avistamientos_df = avistamientos_df[["Latitud","Longitud","Año","Mes","Día","Avistamientos"]]

#Transdormaciones para sacar con fecha (datetime)
avistamientos_fecha_df=avistamientos_df[["Año","Mes","Día"]]
avistamientos_fecha_df.columns = ["year","month","day"]

fecha = pd.to_datetime(avistamientos_fecha_df)

avistamientos_df["Fecha"]=fecha
avistamientos_df=avistamientos_df[["Latitud","Longitud","Fecha","Avistamientos"]]
avistamientos_df.head()

,Latitud,Longitud,Fecha,Avistamientos
3,-25.4,-70.4833,2014-05-01,1
4,-18.4631,-70.3053,2014-05-01,1
5,-27.0658,-70.8259,2014-05-09,13
6,-18.4933,-70.3261,2014-05-10,15
7,-27.122,-70.8576,2014-05-10,23


# Exporto listado de playas sin redondear coordenadas

In [3]:
# Genera excel con las coordenadas de las playas
playas_df = avistamientos_df.reset_index()
playas_df = playas_df[['Latitud','Longitud']]
playas_df.to_excel('./playasSinRedondeoCoord.xlsx')

In [4]:
base = 1/4
def redondeo(x,y):
    return  [base * round(x/base),base * round(y/base)]
x = redondeo(37.1707222222222,-73.2053333333333)
x

[37.25, -73.25]

# Redondeo a cada cuarto de grado
Redondeo de latitud y longitud para juntar lecturas de una misma playa con coordenadas muy proximas

Se exporta el dataframe generado a un excel

In [5]:
base=1/4
def floorQuarter(x):
    return (floor(x * 4) / 4.0) 

def redondeo(x):
    return  base * round(x/base)

#Se añaden atributos con la longitud y latitud redondeadas a cada cuarto de grado
avistamientos_df["Lat_floor"] = avistamientos_df.Latitud.map(redondeo)
avistamientos_df["Long_floor"] = avistamientos_df.Longitud.map(redondeo)

df = avistamientos_df[['Lat_floor', 'Long_floor',"Fecha","Avistamientos"]]
df = df.set_index(['Lat_floor', 'Long_floor',"Fecha"])

#DataFrame total avistamientos de playas en esa cuadricula
df_sum = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).sum()

#DataFrame número de playas en esa cuadricula
df_count = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).count()

#Dataframe con el total de avistamientos y el número de playas
df_join = df_sum.join(df_count,lsuffix="I",rsuffix="R")
df_join.columns=['Suma', 'N_Playas']
df_join['Media_Playas'] = df_join.Suma /df_join.N_Playas

# DataFrame de avistamientos procesado
df_join.to_excel("AvistamientosConRedondeoCoord-025.xlsx")

df_join.head()

Suma  N_Playas  Media_Playas
Lat_floor Long_floor Fecha                                   
-42.75    -74.00     2015-05-27     1         1           1.0
-41.75    -73.75     2014-06-04     1         1           1.0
                     2015-06-10    20         1          20.0
                     2015-06-11   151         1         151.0
          -73.50     2015-05-23     1         1           1.0

# Redondeo con salto entre cuadrantes de dataSet original (0,0833)
Redondeo de latitud y longitud para juntar lecturas de una misma playa con coordenadas muy proximas

Se exporta el dataframe generado a un excel

In [6]:
base=1/12
def floorTwelfth(x):
    return (floor(x * 12) / 12.0)

def redondeo(x):
    return  base * round(x/base)

#Se añaden atributos con la longitud y latitud redondeadas a cada cuarto de grado
avistamientos_df["Lat_floor"] = avistamientos_df.Latitud.map(redondeo)
avistamientos_df["Long_floor"] = avistamientos_df.Longitud.map(redondeo)

df = avistamientos_df[['Lat_floor', 'Long_floor',"Fecha","Avistamientos"]]
df = df.set_index(['Lat_floor', 'Long_floor',"Fecha"])

#DataFrame total avistamientos de playas en esa cuadricula
df_sum = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).sum()

#DataFrame número de playas en esa cuadricula
df_count = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).count()

#Dataframe con el total de avistamientos y el número de playas
df_join = df_sum.join(df_count,lsuffix="I",rsuffix="R")
df_join.columns=['Suma', 'N_Playas']
df_join['Media_Playas'] = df_join.Suma /df_join.N_Playas

# DataFrame de avistamientos procesado
df_join.to_excel("AvistamientosConRedondeoCoord-00833.xlsx")

In [7]:
df_join.head()

Suma  N_Playas  Media_Playas
Lat_floor  Long_floor Fecha                                   
-42.666667 -74.083333 2015-05-27     1         1           1.0
-41.833333 -73.833333 2014-06-04     1         1           1.0
           -73.500000 2015-05-23     1         1           1.0
-41.666667 -73.750000 2015-06-10    20         1          20.0
                      2015-06-11   151         1         151.0

## Exporto listado de coordenadas para utilixarlo en la web

In [8]:
df_web = df_join.reset_index().rename(columns={"Lat_floor": "Latitud", "Long_floor": "Longitud"})[['Latitud','Longitud']]
df_web.to_excel("Playas.xlsx")

# Estructura Buena

In [9]:
# DF con playas
# df_playas = pd.read_excel('./AvistamientosConRedondeoCoord-025.xlsx').reset_index()
df_playas = pd.read_excel('./AvistamientosConRedondeoCoord-00833.xlsx').reset_index()
'''
Las celdas que estaban unidas por tener el mismo valor, tienen valores nulos
por lo que las relleno con us valor correspodiente
'''
df_playas = df_playas.fillna(method= 'ffill',axis=0)
# Cojo las columnas que me interesan
df_playas = df_playas[["Suma","Lat_floor","Long_floor","Fecha"]]
# Renombro las columnas 
df_playas = df_playas.rename(columns={"Suma": "Avistamientos", "Lat_floor": "Latitud", "Long_floor": "Longitud"})
# Ordeno por fecha
df_playas = df_playas.sort_values(by=["Fecha"])

In [10]:
df_playas # DataFrame con avistamientos en cada playa
cuadrantes = { # Numero de cuadrantes que se concatenarán con las playas
    'izquierda': 0 ,
    'arriba': 0,
    'abajo': 0
}
salto = 1/12 # Salto de coordenadas entre los cuadrantes
listado_archivos = os.listdir('../../descargas') # Listo todos los archivos de Copernicus
variables = ['Coord','Profundidad','mlotst','zos','bottomT','thetao','so','uo','vo']

In [11]:
def busca_archivo(fecha):
    texto ='_{}_'.format(str(fecha).split()[0].replace('-',''))
    archivo = [x for x in listado_archivos if str(texto) in x]
    data = xr.open_dataset('../../descargas/{}'.format(archivo[0])) # cargo el archivo
    return data # devuelvo dataset

def dame_datos(latitud,longitud,ds):
    return ds.sel({'latitude':latitud,'longitude': longitud})

def comprueba_datos(latitud,longitud,ds):
    valor = dame_datos(latitud,longitud,ds)
    while math.isnan(valor.mlotst[0]):
        longitud = longitud - salto
        valor = dame_datos(latitud,longitud,ds)
    return latitud,longitud # devuelvo las coordenadas con datos

def crea_DF():
    df_resultados = None
    df_resultados = pd.DataFrame(columns=["Avistamientos","Latitud","Longitud","Fecha",'Coord',"Profundidad","mlotst",'zos','bottomT','thetao','so','uo','vo']) # columnas básicas
    for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda meto un grupo de columnas
        var_aux = [str(x+'_izq_'+str(izquierda+1)) for x in variables]
        df_aux = pd.DataFrame(columns=var_aux)
        df_resultados = df_resultados.join(df_aux)
        for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior meto un grupo de columnas
            var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
        for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior meto un grupo de columnas    
            var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
    return df_resultados

def meto_en_df(df_resultados,row,coordenadas,df,indice,variables_actuales):
    for x in range(3):
        df_resultados.loc[indice + x,["Avistamientos","Latitud","Longitud","Fecha"]] = row.values.tolist() # meto en df_resulados los datos de ese cuadrante
        vars = df.loc[x,['depth', 'mlotst','zos','bottomT','thetao','so','uo','vo']].values.tolist()
        vars.insert(0,coordenadas)
        df_resultados.loc[indice + x,variables_actuales] = vars # meto coordenadas reales utilizadas y datos 
    return df_resultados

def dameCoordIzq(lat,long,n_saltos):
    return [lat,long-(salto*n_saltos)]

def dameCoordSuperior(lat,long,n_saltos):
    coordenadas = dameCoordIzq(lat,long,n_saltos)
    return [lat+(salto*n_saltos),long]

def dameCoordInferior(lat,long,n_saltos):
    return [lat-(salto*n_saltos),long]

In [12]:
def crea_estr():
    df_resultados = crea_DF() # creo df con columnas necesarias
    #print(df_resultados)
    for contador,[index, row] in enumerate(df_playas.iterrows()): # por cada playa
        print(contador,end='\r')
        ds = busca_archivo(row['Fecha']) # busco el archivo de la fecha 
        coordenadas = comprueba_datos(row['Latitud'],row['Longitud'],ds) # miro a ver si el cuadrante tiene datos, sino cojo el siguiente por la izquierda

        df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() # dataframe con los datso oceanicos de esas coordenadas y en esa fecha
        df_resultados = meto_en_df(df_resultados,row,list(coordenadas),df,contador * 3,variables)

        # datos de las celdas extras
        for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda, añado datos
            coordenadas = dameCoordIzq(coordenadas[0],coordenadas[1],izquierda+1)
            df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() 
            var_aux = [x+'_izq_'+str(izquierda+1) for x in variables]
            df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior, añado datos
                coord_aux = dameCoordSuperior(coord_aux[0],coord_aux[1],superior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior, añado datos  
                coord_aux = dameCoordInferior(coord_aux[0],coord_aux[1],inferior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)

#         if contador == 5:
#             break
    return df_resultados

In [13]:
df_resultados = crea_estr()
df_resultados.head()

,Avistamientos,Latitud,Longitud,Fecha,Coord,Profundidad,mlotst,zos,bottomT,thetao,so,uo,vo
0,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.58333333333333]",0.494025,12.5126,0.194708,12.5373,15.8861,34.7331,-0.0592059,-0.14771
1,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.58333333333333]",5.07822,12.5126,0.194708,12.5373,15.8524,34.7331,-0.0640889,-0.175176
2,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.58333333333333]",9.573,12.5126,0.194708,12.5373,15.8209,34.7316,-0.0646992,-0.189215
3,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",0.494025,10.5289,0.219428,15.3323,17.6198,34.8888,-0.0714133,0.136113
4,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",5.07822,10.5289,0.219428,15.3323,17.2711,34.9101,-0.0622578,0.0860622


In [19]:
df_resultados = df_resultados.reset_index().set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad'])
df_resultados.to_excel('EstructuraFinal.xlsx')
df_resultados.head()

a


Coord  \
Latitud    Longitud   Fecha      Avistamientos Profundidad                                             
-25.416667 -70.500000 2014-05-01 1             0.494025     [-25.41666666666666, -70.58333333333333]   
                                               5.078224     [-25.41666666666666, -70.58333333333333]   
                                               9.572997     [-25.41666666666666, -70.58333333333333]   
-18.500000 -70.333333 2014-05-01 1             0.494025                  [-18.5, -70.33333333333333]   
                                               5.078224                  [-18.5, -70.33333333333333]   

                                                             mlotst       zos  \
Latitud    Longitud   Fecha      Avistamientos Profundidad                      
-25.416667 -70.500000 2014-05-01 1             0.494025     12.5126  0.194708   
                                               5.078224     12.5126  0.194708   
                                               9.572997     12.5126  0.194708   
-18.500000 -70.333333 2014-05-01 1             0.494025     10.5289  0.219428   
                                               5.078224     10.5289  0.219428   

                                                            bottomT   thetao  \
Latitud    Longitud   Fecha      Avistamientos Profundidad                     
-25.416667 -70.500000 2014-05-01 1             0.494025     12.5373  15.8861   
                                               5.078224     12.5373  15.8524   
                                               9.572997     12.5373  15.8209   
-18.500000 -70.333333 2014-05-01 1             0.494025     15.3323  17.6198   
                                               5.078224     15.3323  17.2711   

                                                                 so  \
Latitud    Longitud   Fecha      Avistamientos Profundidad            
-25.416667 -70.500000 2014-05-01 1             0.494025     34.7331   
                                               5.078224     34.7331   
                                               9.572997     34.7316   
-18.500000 -70.333333 2014-05-01 1             0.494025     34.8888   
                                               5.078224     34.9101   

                                                                   uo  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-25.416667 -70.500000 2014-05-01 1             0.494025    -0.0592059   
                                               5.078224    -0.0640889   
                                               9.572997    -0.0646992   
-18.500000 -70.333333 2014-05-01 1             0.494025    -0.0714133   
                                               5.078224    -0.0622578   

                                                                   vo  
Latitud    Longitud   Fecha      Avistamientos Profundidad             
-25.416667 -70.500000 2014-05-01 1             0.494025      -0.14771  
                                               5.078224     -0.175176  
                                               9.572997     -0.189215  
-18.500000 -70.333333 2014-05-01 1             0.494025      0.136113  
                                               5.078224     0.0860622

In [20]:
df_resultados_pruebas = df_resultados.copy()
df_resultados_pruebas.drop(columns = 'Coord').set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad'])

mlotst  \
Latitud    Longitud   Fecha      Avistamientos Profundidad            
-25.416667 -70.500000 2014-05-01 1             0.494025     12.5126   
                                               5.078224     12.5126   
                                               9.572997     12.5126   
-18.500000 -70.333333 2014-05-01 1             0.494025     10.5289   
                                               5.078224     10.5289   
...                                                             ...   
-26.333333 -70.666667 2016-03-13 2             5.078224     10.6815   
                                               9.572997     10.6815   
-27.000000 -70.750000 2016-03-13 5             0.494025     10.5289   
                                               5.078224     10.5289   
                                               9.572997     10.5289   

                                                                  zos  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-25.416667 -70.500000 2014-05-01 1             0.494025      0.194708   
                                               5.078224      0.194708   
                                               9.572997      0.194708   
-18.500000 -70.333333 2014-05-01 1             0.494025      0.219428   
                                               5.078224      0.219428   
...                                                               ...   
-26.333333 -70.666667 2016-03-13 2             5.078224     0.0592059   
                                               9.572997     0.0592059   
-27.000000 -70.750000 2016-03-13 5             0.494025     0.0723289   
                                               5.078224     0.0723289   
                                               9.572997     0.0723289   

                                                            bottomT   thetao  \
Latitud    Longitud   Fecha      Avistamientos Profundidad                     
-25.416667 -70.500000 2014-05-01 1             0.494025     12.5373  15.8861   
                                               5.078224     12.5373  15.8524   
                                               9.572997     12.5373  15.8209   
-18.500000 -70.333333 2014-05-01 1             0.494025     15.3323  17.6198   
                                               5.078224     15.3323  17.2711   
...                                                             ...      ...   
-26.333333 -70.666667 2016-03-13 2             5.078224     13.9825  18.7726   
                                               9.572997     13.9825  17.4872   
-27.000000 -70.750000 2016-03-13 5             0.494025     13.1306  19.8596   
                                               5.078224     13.1306  19.6274   
                                               9.572997     13.1306  19.3234   

                                                                 so  \
Latitud    Longitud   Fecha      Avistamientos Profundidad            
-25.416667 -70.500000 2014-05-01 1             0.494025     34.7331   
                                               5.078224     34.7331   
                                               9.572997     34.7316   
-18.500000 -70.333333 2014-05-01 1             0.494025     34.8888   
                                               5.078224     34.9101   
...                                                             ...   
-26.333333 -70.666667 2016-03-13 2             5.078224     34.7255   
                                               9.572997      34.724   
-27.000000 -70.750000 2016-03-13 5             0.494025     34.7026   
                                               5.078224     34.6995   
                                               9.572997      34.695   

                                                                   uo  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-25.416667 -70.500000 2014-05-01 1             0.494025    -0.0592059   
   

## Estructura sin rellenar huecos sin datos

In [ ]:
def crea_estr():
    df_resultados = crea_DF() # creo df con columnas necesarias
    #print(df_resultados)
    for contador,[index, row] in enumerate(df_playas.iterrows()): # por cada playa
        print(contador,end='\r')
        ds = busca_archivo(row['Fecha']) # busco el archivo de la fecha 
        coordenadas = [row['Latitud'],row['Longitud']]

        df = dame_datos(row['Latitud'],row['Longitud'],ds).to_dataframe().reset_index() # dataframe con los datso oceanicos de esas coordenadas y en esa fecha
        df_resultados = meto_en_df(df_resultados,row,[row['Latitud'],row['Longitud']],df,contador * 3,variables)
        
        # datos de las celdas extras
        for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda, añado datos
            coordenadas = dameCoordIzq(coordenadas[0],coordenadas[1],izquierda+1)
            df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() 
            var_aux = [x+'_izq_'+str(izquierda+1) for x in variables]
            df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior, añado datos
                coord_aux = dameCoordSuperior(coord_aux[0],coord_aux[1],superior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior, añado datos  
                coord_aux = dameCoordInferior(coord_aux[0],coord_aux[1],inferior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
        
    return df_resultados

In [ ]:
df_resultados_con_vacios = crea_estr()

In [ ]:
df_resultados_con_vacios.head()

In [ ]:
df_resultados_con_vacios.to_excel('EstructuraConVacios.xlsx')

# Rellenar datos de celdas vacios
Con datos de la coordenada más cercana por la izquierda

In [20]:
def comprueba_datos_thetao(latitud,longitud,ds):
    valor = dame_datos(latitud,longitud,ds)
    while math.isnan(valor.thetao[0][2]):
        longitud = float(longitud) - salto
        valor = dame_datos(latitud,longitud,ds)
    return latitud,longitud # devuelvo las coordenadas con datos

In [54]:
# datos_excel = pd.read_excel('EstructuraFinal.xlsx').reset_index() # leo estructura de datos que contiene huecos sin informacion
# datos = datos_excel.drop(['Unnamed: 0'],axis=1).sort_values(by=['Latitud','Longitud','Profundidad','Fecha']) 
datos_excel = df_resultados.copy().reset_index()
datos = datos_excel.sort_values(by=['Latitud','Longitud','Fecha']) 

In [55]:
datos

,Latitud,Longitud,Fecha,Avistamientos,Profundidad,Coord,mlotst,zos,bottomT,thetao,so,uo,vo
381,-42.666667,-74.083333,2015-05-27,1,0.494025,"[-42.66666666666666, -74.16666666666666]",49.5926,0.106815,12.7754,12.7812,33.3445,0.0347911,-0.0799585
382,-42.666667,-74.083333,2015-05-27,1,5.078224,"[-42.66666666666666, -74.16666666666666]",49.5926,0.106815,12.7754,12.7805,33.3445,0.0286874,-0.0518815
383,-42.666667,-74.083333,2015-05-27,1,9.572997,"[-42.66666666666666, -74.16666666666666]",49.5926,0.106815,12.7754,12.7805,33.3445,0.0238044,-0.0390637
21,-41.833333,-73.833333,2014-06-04,1,0.494025,"[-41.83333333333333, -73.83333333333333]",11.1393,0.0589007,11.646,11.6013,33.5994,0.113529,0.0390637
22,-41.833333,-73.833333,2014-06-04,1,5.078224,"[-41.83333333333333, -73.83333333333333]",11.1393,0.0589007,11.646,11.6284,33.6238,0.107425,0.0341807
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,-18.500000,-70.333333,2014-08-17,1,5.078224,"[-18.5, -70.33333333333333]",10.5289,0.0360118,14.2381,15.1946,34.7621,-0.0305185,0.0799585
227,-18.500000,-70.333333,2014-08-17,1,9.572997,"[-18.5, -70.33333333333333]",10.5289,0.0360118,14.2381,14.8965,34.7667,-0.021363,0.0408948
588,-18.416667,-70.333333,2015-08-18,1,0.494025,"[-18.41666666666666, -70.33333333333333]",14.4963,0.0546281,15.6517,15.6795,34.8582,-0.117801,0.175787
589,-18.416667,-70.333333,2015-08-18,1,5.078224,"[-18.41666666666666, -70.33333333333333]",14.4963,0.0546281,15.6517,15.6429,34.8582,-0.11536,0.151372


In [60]:
def rellenar_huecos(df):
    filas_nan = df[df.isna().any(axis=1)] # filas con valores vacios
    datos_copia = df.copy()
    for indice, fila in filas_nan.iterrows():
#         print(indice,fila)
        elem = fila.isna() # columnas con valores nan
        for index, value in elem.items():
            if value:
                nombre_original= index
                nombre = 'Coord'
                try:
                    nombre_original = index[:index.index('_')] # nombre de la columna en el DataSet
                    nombre = nombre + index[index.index('_'):] # nombre de la columna de coordenadas de ese punto
                except:
                    pass
                coordenadas = fila[nombre]
                archivo = busca_archivo(fila['Fecha'])
                coordenadas = comprueba_datos_thetao(coordenadas[0],coordenadas[1],archivo)
                datos_fila = dame_datos(coordenadas[0],coordenadas[1],archivo)
                valor =datos_fila.sel({'depth':fila['Profundidad']}).to_dataframe().reset_index()[nombre_original].values[0]
                datos_copia.loc[indice,index] = valor
    return datos_copia

In [61]:
datos_sin_huecos = rellenar_huecos(datos)

In [62]:
filas_nan_2 = datos_sin_huecos[datos_sin_huecos.isna().any(axis=1)] # compruebo que se hayan rellenado todos los vacios
filas_nan_2.head()

,Latitud,Longitud,Fecha,Avistamientos,Profundidad,Coord,mlotst,zos,bottomT,thetao,so,uo,vo


In [63]:
datos_sin_huecos.to_excel('EstructuraFinalSinVacios.xlsx')